In [12]:
# importing libraries
import boto3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'webster-data445-bucket'
bucket = s3.Bucket(bucket_name)

# defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

# reading the csv file
heart = pd.read_csv(file_content_stream)
# dropping missing values
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [9]:
# defining the input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

# splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [11]:
# tranforming to 0:1 scaler
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [14]:
# adaboost model
AB = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 3), n_estimators = 500).fit(X_train, Y_train)

pred1 = AB.predict_proba(X_test)[:, 1]
pred1 = np.where(pred1 < 0.1, 0, 1)

# computing recall
recall_score(Y_test, pred1)

1.0

In [13]:
# building the random forest
RF = RandomForestClassifier(n_estimators = 500, max_depth = 3).fit(X_train, Y_train)

# predicting on test
pred2 = RF.predict_proba(X_test)[:, 1]
pred2 = np.where(pred2 < 0.1, 0, 1)

# computing recall
recall_score(Y_test, pred2)

0.9464285714285714

In [15]:
print('The recall score of the Ada Boost Classifier is', recall_score(Y_test, pred1))
print('The recall score of the Random Forest Classifier is', recall_score(Y_test, pred2))

## We would select the Ada Boost classifier because the recall score is higher.

The recall score of the Ada Boost is 1.0
The recall score of the Random Forest is 0.9464285714285714
